<a href="https://colab.research.google.com/github/Kyle950204/201501046KYLE/blob/main/%EC%98%81%ED%99%94_%EB%B6%80%EC%82%B0%ED%96%89_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [1]:
# konlpy 설치하기
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 460kB 43.2MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
# konlpy를 실행하기
import konlpy

In [3]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [4]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
## 문서 ID로 실제 파일 불러오기
# 부산행 58000건.csv
## https://drive.google.com/file/d/1HsC_TXBZHyXXmBO7zpx3Z4_ifQZG5k-j/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1HsC_TXBZHyXXmBO7zpx3Z4_ifQZG5k-j'})
rawdata_downloaded.GetContentFile('부산행 58000건.csv')

In [7]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('부산행 58000건.csv', sep=",")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,박은영(fall****),영화 를 이렇게 세련되게 잘 만들어놓고. 임시완을 다시 보게 만들어놓고. 왜 감독은...,10,3068,396,2017-05-22 18:09:00,관람객
1,1,sj(aban****),"대선때문에 영화 홍보가 안되고 대선 미루라고 하며, 더킹과 마스터를 조무래기라고 리...",1,4547,1970,2017-05-18 17:23:00,0
2,2,미짱씨(skad****),불한당 기대했는데 감독때문에 안볼것입니다 감독이 전라도비하 여성비하를 리트윗했다죠?...,1,4336,1760,2017-05-18 17:26:00,0
3,3,pumpkin EyE(lily****),감독만큼 수준 이하임. 이 감독 트윗한번 보고 나니. 왜 이따위인지 알겠음.,1,4557,2007,2017-05-18 17:25:00,0
4,4,앨리스(aeon****),감독 문제로 보이콧하기에는 너무 잘만든 영화. 간만에 보는 웰메이드 느와르.,10,2814,402,2017-05-22 19:06:00,관람객
...,...,...,...,...,...,...,...,...
12575,12575,핫포(csp7****),솔직히. 임시완이 연기는 잘하지만. 이영화의. 역활에는 좀 붕뜨지안나요? 어느정도...,6,1,25,2018-04-26 14:45:00,0
12576,12576,히끗ㅜㅜ(wd86****),영화 속 캐릭터들 하나 하나가 어찌나 임팩트 있던지 완전 딱 내스탈!! 기회되면 한...,10,2,26,2017-05-19 05:14:00,0
12577,12577,FormerAction(form****),감독도 문제이지만 오로지 팬덤많으로 관객수 90만찍는것도 참 한심,1,5,29,2017-06-07 00:25:00,0
12578,12578,kyun****,볼만한영화추천합니다,10,12,36,2017-05-19 12:43:00,관람객


In [8]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [9]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,순수인(kjni****),좀비가 무서운게아니라 사람이무섭다는걸 알려주는영화,10,19664,1253,2016-07-20 05:57:00,0
1,1,20fh****,그냥 한국형 좀비물이 어색하지 않았다는 거에 큰 점수를 주며 앞으로의 한국영화를 기...,10,14654,1441,2016-07-20 11:56:00,관람객
2,2,ㄹㅇㅁㄻㅇ(wjse****),버스회사 상무라는 사람이 진심 역대급 민폐캐릭터인듯개짜증나고 생존력도 좋음,10,13845,1061,2016-07-20 00:00:00,0
3,3,kpl(lkp7****),생각보다 너무 재밌었다,10,12444,1950,2016-07-20 01:11:00,관람객
4,4,쥘(sgre****),영화가 너무 비현실적이네요. 예를들어 공유 얼굴이라던가 공유 얼굴 같은 부분이...,6,12594,2539,2016-07-20 07:30:00,0
...,...,...,...,...,...,...,...,...
57995,57995,hiorangNa(anna****),태어나서 평점처음남김태어나서 이렇게 웃긴영화는처음 기가차는소리 5분에 한번씩함ㅋOO...,1,5,11,2016-08-05 02:33:00,0
57996,57996,knev****,10분 관람하다 급한일생겨 나옴...,1,5,11,2016-08-02 23:27:00,관람객
57997,57997,lemo****,"너무많이 기대를 해서인지 실망감이 있네요,뭔가 허무한 엔딩",1,5,11,2016-07-31 22:16:00,관람객
57998,57998,방지현(step****),공유 얼굴 제외 볼게없음,1,6,12,2016-08-16 07:45:00,0


# 형태소 분석

In [10]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [11]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [12]:
# "형태소" 변수의 저장 내용 확인
형태소

[('내용', 'Noun'),
 ('도', 'Josa'),
 ('없고', 'Adjective'),
 ('참', 'Verb'),
 ('한심한', 'Adjective'),
 ('영화', 'Noun')]

In [13]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,내용,Noun,1,1
1,도,Josa,1,1
2,없고,Adjective,1,1
3,참,Verb,1,1
4,한심한,Adjective,1,1
5,영화,Noun,1,1


In [14]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[       0          1  type  count
 0     좀비       Noun    10      1
 1      가       Josa    10      1
 2   무서운게  Adjective    10      1
 3    아니라  Adjective    10      1
 4     사람       Noun    10      1
 5      이       Josa    10      1
 6  무섭다는걸  Adjective    10      1
 7   알려주는       Verb    10      1
 8     영화       Noun    10      1,        0            1  type  count
 0     그냥         Noun    10      1
 1     한국         Noun    10      1
 2      형       Suffix    10      1
 3     좀비         Noun    10      1
 4     물이         Noun    10      1
 5   어색하지    Adjective    10      1
 6   않았다는         Verb    10      1
 7      거         Noun    10      1
 8      에         Josa    10      1
 9      큰         Verb    10      1
 10    점수         Noun    10      1
 11     를         Josa    10      1
 12    주며         Verb    10      1
 13     앞         Noun    10      1
 14   으로의         Josa    10      1
 15   한국영         Noun    10      1
 16     화       Suffix    10      1
 17     를   

In [19]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [20]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,좀비,Noun,10,1
1,가,Josa,10,1
2,무서운게,Adjective,10,1
3,아니라,Adjective,10,1
4,사람,Noun,10,1
...,...,...,...,...
1,도,Josa,1,1
2,없고,Adjective,1,1
3,참,Verb,1,1
4,한심한,Adjective,1,1


In [21]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [22]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,좀비,Noun,10,1
1,가,Josa,10,1
2,무서운게,Adjective,10,1
3,아니라,Adjective,10,1
4,사람,Noun,10,1
...,...,...,...,...
1,도,Josa,1,1
2,없고,Adjective,1,1
3,참,Verb,1,1
4,한심한,Adjective,1,1


In [23]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [24]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소    품사           분류
      Foreign      6      1
     Foreign      10     1
      Foreign      6      1
                    10     2
!      Punctuation  1     90
                          ..
？      Foreign      1      1
＼      Foreign      10     1
｀ω´    Foreign      10     1
～～～～。  Foreign      10     1
￦￦★♥   Foreign      10     1
Name: 카운트, Length: 104945, dtype: int64

In [25]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [26]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소   품사          분류     
     Foreign     6     1
    Foreign     10    1
     Foreign     6     1
                  10    2
!     Punctuation 1    90
...                   ...
？     Foreign     1     1
＼     Foreign     10    1
｀ω´   Foreign     10    1
～～～～。 Foreign     10    1
￦￦★♥  Foreign     10    1

[104945 rows x 1 columns]

In [27]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')